In [8]:
from qutip import *

import numpy as np

import matplotlib.pyplot as plt

from numba import jit, complex128

from scipy.linalg import expm

In [18]:
def Hxyz(N, Jx, Jy, Jz, b, periodic=False):
    """Generates an XYZ spin chain Hamiltonian with N spins"""
    J_list = [Jx, Jy, Jz]
    sigx = np.array([[0, 1. + 0j], [1., 0]])
    sigy = np.array([[0, -1j], [1j, 0]])
    sigz = np.array([[1. + 0j, 0], [0, -1.]])
    sigma_list = [sigx, sigy, sigz]
    if N <= 1:
        return np.identity(2**N)
    Ham = 0
    for i in range(N-1):
        for j in range(3):
            spin_mat = np.identity(1)
            for k in range(N):
                if k == i or k == i+1:
                    spin_mat = np.kron(spin_mat, sigma_list[j])
                else:
                    spin_mat = np.kron(spin_mat, np.identity(2))
            Ham += J_list[j]*spin_mat
    for i in range(N):
        spin_mat = np.identity(1)
        for k in range(N):
            if k == i:
                spin_mat = np.kron(spin_mat, sigz)
            else:
                spin_mat = np.kron(spin_mat, np.identity(2))
        Ham += b*spin_mat
    if periodic:
        for j in range(3):
            spin_mat = np.identity(1)
            for k in range(N):
                if k == 0 or k == N-1:
                    spin_mat = np.kron(spin_mat, sigma_list[j])
                else:
                    spin_mat = np.kron(spin_mat, np.identity(2))
            Ham += J_list[j]*spin_mat
    return Ham

In [3]:
@jit(nopython=True)
def manual_trace(matrix):
    trace = 0.0 + 0.0j  # Explicitly specifying as complex
    for i in range(matrix.shape[0]):
        trace += matrix[i, i]
    return trace

@jit(nopython=True)
def partial_trace_final(state, final_subsys_size, remainder_size):
    if state.ndim != 2 or state.shape[0] != final_subsys_size * remainder_size:
        raise ValueError("Invalid state dimensions")

    reshaped_state = state.reshape((remainder_size, final_subsys_size, remainder_size, final_subsys_size))
    
    # Manual trace computation with consistent data types
    traced_state = np.zeros((remainder_size, remainder_size), dtype=np.complex128)
    for i in range(remainder_size):
        for j in range(remainder_size):
            traced_state[i, j] = manual_trace(reshaped_state[i, :, j, :])

    normalization = manual_trace(traced_state)
    if normalization != 0:
        traced_state /= normalization

    return traced_state

In [100]:
def collision_loop(state, rho_A, U_S,  U_SA, expec_list_S, num_col, t, gamma, dt):
    expec_list_SA = [np.kron(ex, np.identity(2)) for ex in expec_list_S]
    free_steps = np.ceil(np.random.exponential(gamma/dt, size = num_col))
    col_step = int(np.ceil(t/dt))
    total_col = np.sum(free_steps) + col_step*num_col
    num_expec = len(expec_list_S)
    expec_out = np.zeros((int(total_col), num_expec))
    rho_SA = state
    N_S = U_S.shape[0]
    N_A = rho_A.shape[0]
    expec_loc = 0

    for i in range(num_col):
        for _ in range(col_step):
            rho_SA = U_SA @ rho_SA @ U_SA.conj().T
            for k in range(num_expec):
                expec_out[expec_loc][k] = np.trace(rho_SA @ expec_list_SA[k])
            expec_loc += 1
        
        rho_S = partial_trace_final(rho_SA, N_A, N_S)

        for _ in range(int(free_steps[i])):
            rho_S = U_S @ rho_S @ U_S.conj().T
            for k in range(num_expec):
                expec_out[expec_loc][k] = np.trace(rho_S @ expec_list_S[k])
            expec_loc += 1
        
        rho_SA = np.kron(rho_S, rho_A)

    return expec_out


def collision_model_dynamics(num_col, rho_S_0, rho_A, expec_list_S, H_S, H_int, t, gamma, dt):
    """Simulates collision model dynamics and returns expectation values at each step"""
    state = np.kron(rho_S_0, rho_A)
    H_col = np.kron(H_S, np.identity(2)) + H_int
    
    U_S = expm(-1j * H_S * dt)
    U_SA = expm(-1j * H_col * dt)
    
    return collision_loop(state, rho_A, U_S,  U_SA, expec_list_S, num_col, t, gamma, dt)

In [4]:
# Sample 4x4 density matrix for a 2-qubit system
state = np.array([[0.5, 0.0, 0.0, 0.5],
                  [0.0, 0.0, 0.0, 0.0],
                  [0.0, 0.0, 0.0, 0.0],
                  [0.5, 0.0, 0.0, 0.5]])

# Subsystems to trace out (second qubit)
subsystems = [1]

# Dimension list for each subsystem (2-qubit system)
dim_list = [2, 2]

# Now you can call the partial_trace function with these inputs
state2 = np.array([1/2, -1j/2, 1j/2, 1/2])
result1 = partial_trace_final(state, 2, 2)
print(result1)

[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]


In [102]:
#set all the relevant parameters
bfield = 0.5
J = 1.
gamma = 1.
Delta = 0.

num_collision = 400
collision_duration = 0.5
timestep = 0.01

num_spins = 3

In [103]:
#generate initial states
np.random.seed(10) #10 best for 4 spins
N = 2**num_spins
psi_0 = np.random.rand(N) + 1j * np.random.rand(N)
rho_S_0 = np.outer(psi_0, psi_0)
rho_S_0 /= np.trace(rho_S_0)

beta = 1. #temperature
g_prob = 0.5*(1+np.tanh(beta))
psi_e = [0, 1 + 0j]
psi_g = [1 + 0j, 0]
rho_A = g_prob * np.outer(psi_g, psi_g) + (1-g_prob) * np.outer(psi_e, psi_e)

#expectation values to calculate

sigx = np.array([[0, 1. + 0j], [1., 0]])
sigy = np.array([[0, -1j], [1j, 0]])
sigz = np.array([[1. + 0j, 0], [0, -1.]])

expec_list_X = []
for i in range(num_spins):
    exp_mat = np.identity(1)
    for k in range(num_spins):
        if k == i:
            exp_mat = np.kron(exp_mat, sigx)
        else:
            exp_mat = np.kron(exp_mat, np.identity(2))
    
    expec_list_X.append(exp_mat)

expec_list_Y = []
for i in range(num_spins):
    exp_mat = np.identity(1)
    for k in range(num_spins):
        if k == i:
            exp_mat = np.kron(exp_mat, sigy)
        else:
            exp_mat = np.kron(exp_mat, np.identity(2))
    
    expec_list_Y.append(exp_mat)

expec_list_S = expec_list_X + expec_list_Y

expec_list_SA = [np.kron(ex, np.identity(2)) for ex in expec_list_S]



#system and interaction Hamiltonians
H_S = -Hxyz(num_spins, J, J*gamma, J*Delta, bfield, periodic=True)
H_SA = 0.5*(np.kron(np.identity(N), sigx)
          + np.kron(np.identity(N), sigy))

U_S = expm(-1j * H_S * timestep)
H_col = np.kron(H_S, np.identity(2)) + H_SA
U_SA = expm(-1j * H_col * timestep)

state = np.kron(rho_S_0, rho_A)

rho_S = partial_trace_final(state, 2, N)

In [104]:
expectations = collision_model_dynamics(num_collision, rho_S_0, rho_A, expec_list_S, 
                               H_S, H_SA, collision_duration, gamma, timestep)

C:\Users\35386\AppData\Local\Temp\ipykernel_31400\1378776391.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  expec_out[expec_loc][k] = np.trace(rho_SA @ expec_list_SA[k])
C:\Users\35386\AppData\Local\Temp\ipykernel_31400\1378776391.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  expec_out[expec_loc][k] = np.trace(rho_S @ expec_list_S[k])
